In [ ]:
import requests
import os
import json
import pandas as pd

# ADD TWITTER API BEARER TOKEN
bearer_token = ''
headers = {'Authorization':('Bearer '+ bearer_token)}

# queries tweets including nba/basketball

url = f'https://api.twitter.com/2/tweets/search/recent?query=nba%20basketball&max_results=10'

url += f'&user.fields=created_at,description,entities,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld'
url += f'&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld'
url += f'&expansions=attachments.poll_ids,attachments.media_keys,author_id,geo.place_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id'
url += f'&media.fields=duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width'
url += f'&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type'
url += f'&poll.fields=duration_minutes,end_datetime,id,options,voting_status'

response = requests.request("GET", url, headers=headers)
json_data = json.loads(response.text)
clean_data = json.dumps(json_data, indent=2, sort_keys=True) 
print(clean_data)

{
  "data": [
    {
      "author_id": "1045425205752082437",
      "context_annotations": [
        {
          "domain": {
            "description": "Television shows from around the world",
            "id": "3",
            "name": "TV Shows"
          },
          "entity": {
            "description": "All the latest basketball action from the NBA.",
            "id": "10000607734",
            "name": "NBA Basketball"
          }
        },
        {
          "domain": {
            "description": "Television shows from around the world",
            "id": "3",
            "name": "TV Shows"
          },
          "entity": {
            "description": "Live coverage from the National Basketball Association, the top basketball league in North America.",
            "id": "10029190505",
            "name": "Live: NBA Basketball"
          }
        },
        {
          "domain": {
            "description": "Television shows from around the world",
            "id": "3",
    

In [ ]:
# print dictionary keys

for item in dict(json_data).items():
  print(item[0])
  for sub_item in item[1]:
    if type(sub_item) == dict:
      print(f'  {sub_item.keys()}')
    else:
      print(f'  {sub_item}')

data
  dict_keys(['id', 'context_annotations', 'public_metrics', 'entities', 'author_id', 'text', 'created_at', 'reply_settings', 'conversation_id', 'possibly_sensitive', 'referenced_tweets', 'source', 'lang'])
  dict_keys(['id', 'context_annotations', 'public_metrics', 'entities', 'author_id', 'text', 'created_at', 'reply_settings', 'conversation_id', 'possibly_sensitive', 'referenced_tweets', 'source', 'lang'])
  dict_keys(['id', 'context_annotations', 'public_metrics', 'entities', 'author_id', 'text', 'created_at', 'reply_settings', 'conversation_id', 'possibly_sensitive', 'referenced_tweets', 'source', 'lang'])
  dict_keys(['id', 'context_annotations', 'entities', 'public_metrics', 'author_id', 'text', 'created_at', 'reply_settings', 'conversation_id', 'possibly_sensitive', 'referenced_tweets', 'source', 'lang'])
  dict_keys(['id', 'context_annotations', 'public_metrics', 'entities', 'author_id', 'text', 'created_at', 'reply_settings', 'conversation_id', 'possibly_sensitive', 'refe

In [ ]:
# Print out tweets with id, text, retweets, likes, and if its possibly sensitive

for tweet in json_data['data']:
  print(f"id: \t\t{tweet['id']}")
  print(f"text: \t\t{tweet['text']}")
  print(f"retweets: \t{tweet['public_metrics']['retweet_count']}")
  print(f"likes: \t\t{tweet['public_metrics']['like_count']}")
  print(tweet['possibly_sensitive'])
  
  # check for photo attachment
  media_key = "" 
  if 'attachments' in tweet:
    if 'media_keys' in tweet['attachments']:
      media_key = tweet['attachments']['media_keys'][0]

  if media_key != "":
    # If there is a media key in this tweet, iterate through tweet['includes']['media'] until we find it
    for media in json_data['includes']['media']:
      if media['media_key'] == media_key: # Only if the media_key matches the one we stored
        if media['type'] == 'photo':      # Only if it is a photo; ignore videos
          image_url = media['url']
          print(f"photo: \t\t{image_url}")
  # for i in json_data['includes']['places']:
  #   print(i['country'])

id: 		1442955762305626113
text: 		RT @OccupyDemocrats: BREAKING NEWS: Basketball legend Kareem Abdul-Jabbar demands that the NBA “remove all unvaccinated players and coaches…
retweets: 	16955
likes: 		0
False
id: 		1442955713068761091
text: 		RT @NBCSports: Is a hot dog a sandwich? 🌭

The U.S. Olympic men's basketball team was asked this during the #TokyoOlympics and we're still…
retweets: 	16
likes: 		0
False
id: 		1442955706559131648
text: 		RT @CNN: "The NBA should insist that all players and staff are vaccinated or remove them from the team," basketball Hall of Famer Kareem Ab…
retweets: 	505
likes: 		0
False
id: 		1442955513818337282
text: 		I commented on the initial tweet. If an NBA is loud and wrong they need to be corrected. Not skipped over and asked basketball question.
YOU'RE WELCOME https://t.co/ez5ZvrljAy
retweets: 	0
likes: 		0
False
id: 		1442955494985920515
text: 		RT @OccupyDemocrats: BREAKING NEWS: Basketball legend Kareem Abdul-Jabbar demands that the NBA “remove al

In [ ]:
n = 1000                          # The total number of tweets we want
max_results = 100                 # The number of tweets to pull per request; must be between 10 and 100
total_retrieved = 0               # Keeps track of when to stop
next_token = ""                   # Must be empty on first iteration
search_term = "(nba%20OR%20basketball)"             # To form an advanced query, see here: https://twitter.com/search-advanced?lang=en

df = pd.DataFrame(columns=['id', 'retweets', 'likes', 'replies', 'location', 'language', 'possibly sensitive', 'image_url', 'text'])
df.set_index('id', inplace=True)

while total_retrieved < n:

  if next_token == "":
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}'
  else:
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&next_token={next_token}'

  url += f'&user.fields=created_at,description,entities,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld'
  url += f'&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld'
  url += f'&expansions=attachments.poll_ids,attachments.media_keys,author_id,geo.place_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id'
  url += f'&media.fields=duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width'
  url += f'&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type'
  url += f'&poll.fields=duration_minutes,end_datetime,id,options,voting_status'

  response = requests.request("GET", url, headers=headers)
  try:
    json_data = json.loads(response.text)
  except:
    print(response.text)

  for tweet in json_data['data']:
    media_key = ""

    tweet_id = tweet['id']
    retweet_count = tweet['public_metrics']['retweet_count']
    like_count = tweet['public_metrics']['like_count']
    reply_count = tweet['public_metrics']['reply_count']
    location = ""
    # country = ""
    language = tweet['lang']
    possibly_sensitive = tweet['possibly_sensitive']
    image_url = ""
    text = tweet['text']

    author_id = tweet['author_id']
    if author_id != "":
      for user in json_data['includes']['users']:
        if (author_id == user['id']):
          try:
            location = user['location']
          except:
            break
            
    # location_id = ""
    # if 'geo' in tweet:
    #   location_id = tweet['geo']['place_id']
    #   for i in json_data['includes']['places']:
    #     if location_id == i['id']:
    #       country = i['country']

    # check if tweet has an attachment, only add to dataframe if tweet includes a photo

    if 'attachments' in tweet:
      if 'media_keys' in tweet['attachments']:
        media_key = tweet['attachments']['media_keys'][0]

    if media_key != "":
      for media in json_data['includes']['media']:
        if media['media_key'] == media_key:
          if media['type'] == 'photo':     
            image_url = media['url']        
            
            if location != "":
              total_retrieved += 1
              df.loc[tweet_id] = [retweet_count, like_count, reply_count, location, language, possibly_sensitive, image_url, text]
              break

  try:
    next_token = json_data['meta']['next_token']
  except:
    break  

# Export dataframe to csv file
print(f'Number of records:\t{len(df)}')
df.to_csv('twitter_data.csv')
df

Number of records:	1000


,retweets,likes,replies,location,language,possibly sensitive,image_url,text
id,,,,,,,,
1442957035163688961,0,0,0,"Chez Draxxo, France",fr,False,https://pbs.twimg.com/media/FAZpxVFVgAAkra4.jpg,@GaelArcuset @TheDailyDunkfr @NBA A un tweet d...
1442957025114144768,176,0,0,"Denver, CO",en,False,https://pbs.twimg.com/media/FAZL9y8VUAYmLVa.jpg,RT @AuxGod_: NBA media day with outdated techn...
1442957023247622145,1298,0,0,"Dallas, TX",en,False,https://pbs.twimg.com/media/FAY1F96UcAYnsQJ.jpg,RT @NBA: #6 for year 19! #NBAMediaDay https://...
1442957013881802755,2,0,0,United States,en,False,https://pbs.twimg.com/media/FAZPQIuVcAAPA8p.jpg,"RT @BRAVESSPORTS: Bussie, Alcorn State agree t..."
1442957004792729600,1,2,0,Northwest Arkansas,en,False,https://pbs.twimg.com/media/FAZpvc5UYAEPSYg.jpg,We’re getting a sneak peek at the Arkansas men...
...,...,...,...,...,...,...,...,...
1442939349813788673,131,0,0,A circumscribed square of Hell,en,False,https://pbs.twimg.com/media/FAZX0ZfVUAA7D1F.jpg,"RT @MikePradaNBA: NBA spacing, 2001-present ht..."
1442939343727783937,131,0,0,"Richmond, VA",en,False,https://pbs.twimg.com/media/FAZX0ZfVUAA7D1F.jpg,"RT @MikePradaNBA: NBA spacing, 2001-present ht..."
1442939324421472259,131,0,0,Chicago - Dallas,en,False,https://pbs.twimg.com/media/FAZX0ZfVUAA7D1F.jpg,"RT @MikePradaNBA: NBA spacing, 2001-present ht..."
